#**Aula 3 - Leyendo Archivos JSON**

##Importando Archivos JSON (JAVA SCRIPT OBJECT NOTATION)

Permite escribir los datos de forma ligera para trasmitir de un software, sistemas, aplicativos a otro  por APIS. Es un viaje ligero.

Herramienta nube para ver mejor contenido archivo JSON / Visualizador de JSON : https://jsonviewer.stack.hu/

In [1]:
import pandas as pd

In [2]:
datos_pacientes = pd.read_json('pacientes.json')

FileNotFoundError: File pacientes.json does not exist

In [ ]:
datos_pacientes

In [ ]:
datos_pacientes_2 = pd.read_json('pacientes_2.json')

In [ ]:
datos_pacientes_2
# Este archivo tiene columnas anidadas (un  diccionario dentro de la columna), y debemos usar otro parametro para poder ver correctamente la información.

In [ ]:
# Otro visualizador de JSON : JsonCrack
# Link : https://jsoncrack.com/

##Normalizando Archivos JSON

In [ ]:
# Entrara y resolver el problema de la parte anidada
# funcion json_normalize extrae diccionarios anidados, dentro de alguna columna y separarlo en nuevas variables, haciendo que el analisis sea mas facil de entender
df_normalizado = pd.json_normalize(datos_pacientes_2['Pacientes'])

In [ ]:
df_normalizado

##Escribiendo (Exportando) Archivos JSON

In [ ]:
df_normalizado.to_json('historico_pacientes_norm.json')

In [ ]:
df_normalizado.to_csv('historico_pacientes_norm.csv')

In [ ]:
df_normalizado.to_excel('historico_pacientes_norm.xlsx')

In [ ]:
pd.read_json('historico_pacientes_norm.json')
# no es necesario poner index = false, solo en el CSV

API

In [ ]:
import requests
import json

In [ ]:
datos_frutas = requests.get('https://fruityvice.com/api/fruit/all')

In [ ]:
datos_frutas.json()

In [ ]:
resultado = json.loads(datos_frutas.text)

In [ ]:
pd.DataFrame(resultado)

In [ ]:
# 1 . Crear Banco de Datos Local, el motor que va a soportar mis tablas. Banco de datos que ya viene instalado en Google Colab 'SQLite', no se necesita instalar otro paquete externo ni instalar nada mas. Existen otros MySql, Oracle, Sql Server etc.
#selecciono el motor y digo donde quiero que se ejecute en este caso en la memoria de google colab
#almacenarla en algun lugar para relacionarlo un parametro engine en este caso
engine = create_engine('sqlite:///:memory:')

In [ ]:
archivo = 'clientes_banco.csv'

In [ ]:
datos = pd.read_csv(archivo)

In [ ]:
type(datos)

In [ ]:
datos.head()

In [ ]:
datos.tail()

Necesito que esten los datos en una variable Data Frame para pasarlo a la base de datos, no puedo pasarlo directo.

In [ ]:
#crear tabla clientes con los datos del Data Frame datos
datos.to_sql('clientes',engine, index=False)

In [ ]:
# Validar que creo la tabla clientes, parametro que reciba la ejecucion del inspector
inspector = inspect(engine)

In [ ]:
print(inspector.get_table_names())
# Aqui observo que ya hay una tabla con el nombre clientes

##Leyendo una consulta SQL

In [ ]:
# filtar solo clientes con categoria de renta = empleado
query = 'SELECT * FROM clientes WHERE Categoria_de_renta = "Empleado"'

In [ ]:
# llevar la consulta a SQL, necesario envolverlo en text el query, y poner con de conexion para el motor
empleados = pd.read_sql(sql = text(query), con = engine)

In [ ]:
type(empleados)

In [ ]:
# verific que quedo en el data fream empleado
empleados

In [ ]:
empleados.to_sql('empleados', con = engine.connect(), index = False)

read.sql, es para leer una parte de la tabla pero no la tabla completa, si quiero leer la tabla completa , puedo usar

In [ ]:
#pd.read_sql
pd.read_sql_table('empleados', con = engine.connect())

In [ ]:
# consultar tabla completa
pd.read_sql_table('empleados', con = engine.connect(), columns = ['ID_Cliente','Grado_estudio','Rendimiento_anual'])

##Actalizando un banco de datos

In [ ]:
query = 'SELECT * FROM clientes'

In [ ]:
#ejecutar query
pd.read_sql(sql = text(query), con = engine.connect())

###Borrar registros

In [ ]:
#Borrando registros
#Importar biblioeta de sqlalquemy para saber que tipo de error es en caso de suceder
from sqlalchemy.exc import SQLAlchemyError
#Borrar en SQL
query = "DELETE FROM clientes WHERE ID_Cliente = 5008804 "
try:
  #ejecuta setencia DELET no devuelve registros
  r_set = engine.connect().execute(text(query))
except SQLAlchemyError as e:
   print(e)
else:
   print("#Registros borrados: ", r_set.rowcount)

In [ ]:
pd.read_sql_table('clientes', con = engine.connect())

###Actualizar registros

In [ ]:
#Actualizando registros
query = "UPDATE clientes SET Grado_estudio = 'Nivel superior' WHERE ID_Cliente = 5008808"
try:
  #ejecuta setencia UPDATE no devuelve registros
  r_set = engine.connect().execute(text(query))
except SQLAlchemyError as e:
   print(e)
else:
   print("#Registros aCtualizados: ", r_set.rowcount)

In [ ]:
pd.read_sql_table('clientes', con = engine.connect())

In [ ]:
inspector = inspect(engine)
print(inspector.get_table_names())